In [1]:
# -*- coding: utf-8 -*-

from __future__ import print_function
import tensorflow as tf

tf.reset_default_graph()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
print("Download Done!")

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Download Done!


In [2]:
# parameters
learning_rate  = 1e-3
training_iters = 100000
batch_size     = 50
display_step   = 100
save_path      = None
logs_path      = "TensorBoard/lenet"
model_path     = "Model/lenet"

# network parameters
n_input   = 784  # input data (img shape: 28 * 28)
n_classes = 10   # total classes (0-9 digits)
dropout   = 0.75 # dropout, probability to keep units

# tf graph input
x = tf.placeholder(tf.float32, [None, n_input], name = 'X')
y = tf.placeholder(tf.float32, [None, n_classes], name = 'Y')
keep_prob = tf.placeholder(tf.float32, name = "keep_prob") #dropout (keep probability)

In [3]:
# create some wrappers for simplicity
def weight_variable(shape, name):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial, name = name)

def bias_variable(shape, name):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial, name = name)

def conv2d(x, W, b, strides = 1):
    x = tf.nn.conv2d(x, W,                               \
                     strides = [1, strides, strides, 1], \
                     padding = "SAME")
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def max_pool(x, k = 2):
    return tf.nn.max_pool(x,                      \
                          ksize   = [1, k, k, 1], \
                          strides = [1, k, k, 1], \
                          padding = "SAME")

In [4]:
weights = {
    # 5 x 5 conv,  1 input , 32 outputs
    "W_conv1": weight_variable([5, 5, 1, 32], name = "W_conv1"),
    # 5 x 5 conv, 32 inputs, 64 outputs
    "W_conv2": weight_variable([5, 5, 32, 64], name = "W_conv2"),
    # 7 * 7 * 64 inputs, 1024 outputs
    "W_fc1": weight_variable([7 * 7 * 64, 1024], name = "W_fc1"),
    # 1024 inputs, 10 outputs (class prediction)
    "W_out": weight_variable([1024, n_classes], name = "W_out")
}

biases = {
    "b_conv1": bias_variable([32], name = "b_conv1"),
    "b_conv2": bias_variable([64], name = "b_conv2"),
    "b_fc1": bias_variable([1024], name = "b_fc1"),
    "b_out": bias_variable([n_classes], name = "b_out")
}

In [5]:
# create model
def lenet(x, weights, biases, dropout):
    # reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # conv layer-1
    h_conv1 = conv2d(x, weights["W_conv1"], biases["b_conv1"])
    h_pool1 = max_pool(h_conv1, k = 2)

    # conv layer-2
    h_conv2 = conv2d(h_pool1, weights["W_conv2"], biases["b_conv2"])
    h_pool2 = max_pool(h_conv2, k = 2)

    # full connection
    # reshape conv2 output to fit fully connected layer input
    h_pool2_flat = tf.reshape(h_pool2, [-1, weights["W_fc1"].get_shape().as_list()[0]])
    h_fc1 = tf.nn.relu(tf.add(tf.matmul(h_pool2_flat, weights["W_fc1"]), biases["b_fc1"]))

    # apply dropout
    h_fc1_drop = tf.nn.dropout(h_fc1, dropout)

    # output layer, class prediction
    out = tf.add(tf.matmul(h_fc1_drop, weights["W_out"]), biases["b_out"])
    return out

In [6]:
def get_pred(x, weights, biases, keep_prob):
    return lenet(x, weights, biases, keep_prob)

def get_cost(y, y_hat):
    cost = tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = y_hat)
    return tf.reduce_mean(cost)

def get_accuracy(y, y_hat):
    correct_pred = tf.equal(tf.argmax(y_hat, 1), tf.argmax(y, 1))
    return tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [7]:
# build model
y_hat = tf.identity(get_pred(x, weights, biases, keep_prob), "Y_hat")

# define loss and optimizer
cost = tf.identity(get_cost(y, y_hat), "cost")
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

# evaluate model
accuracy = tf.identity(get_accuracy(y, y_hat), "accuracy")

tf.summary.scalar("loss", cost)
tf.summary.scalar("accuracy", accuracy)

merged_summary_op = tf.summary.merge_all()

# 'Saver' op to save and restore all the variables
saver = tf.train.Saver()

In [8]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter(logs_path, graph = tf.get_default_graph())

    step = 1

    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})

        if step % display_step == 0:
            # calculate batch loss and accuracy
            loss, acc, summary = sess.run([cost, accuracy, merged_summary_op], \
                                          feed_dict = {x: batch_x, y: batch_y, \
                                                       keep_prob: 1.})

            summary_writer.add_summary(summary, step * batch_size)

            print("Iter              : %d  " %(step * batch_size))
            print("Minibatch Loss    : %.2f" %(loss))
            print("Training  Accuacy : %.2f" %(acc), '\n')

        step += 1

    print("Optimization Finished!")

    print("Testing Accuracy : %.2f" %(sess.run(accuracy, feed_dict =  \
                                               {x: mnist.test.images, \
                                                y: mnist.test.labels, \
                                                keep_prob: 1.})), '\n')

    # Save model weights to disk
    save_path = saver.save(sess, model_path, global_step = step * batch_size)
    print("Model saved in file: %s" % save_path)

Iter              : 5000  
Minibatch Loss    : 0.07
Training  Accuacy : 0.98 

Iter              : 10000  
Minibatch Loss    : 0.08
Training  Accuacy : 0.96 

Iter              : 15000  
Minibatch Loss    : 0.05
Training  Accuacy : 0.98 

Iter              : 20000  
Minibatch Loss    : 0.04
Training  Accuacy : 1.00 

Iter              : 25000  
Minibatch Loss    : 0.02
Training  Accuacy : 1.00 

Iter              : 30000  
Minibatch Loss    : 0.23
Training  Accuacy : 0.96 

Iter              : 35000  
Minibatch Loss    : 0.02
Training  Accuacy : 1.00 

Iter              : 40000  
Minibatch Loss    : 0.03
Training  Accuacy : 1.00 

Iter              : 45000  
Minibatch Loss    : 0.06
Training  Accuacy : 0.96 

Iter              : 50000  
Minibatch Loss    : 0.01
Training  Accuacy : 1.00 

Iter              : 55000  
Minibatch Loss    : 0.02
Training  Accuacy : 1.00 

Iter              : 60000  
Minibatch Loss    : 0.01
Training  Accuacy : 1.00 

Iter              : 65000  
Minibatch Los

In [9]:
def ckpt_test(graph):
    x = graph.get_tensor_by_name("X:0")
    y = graph.get_tensor_by_name("Y:0")
    keep_prob = graph.get_tensor_by_name("keep_prob:0")
    accuracy  = graph.get_tensor_by_name("accuracy:0")
    
    print("Testing Accuracy : %.2f" %(sess.run(accuracy, feed_dict =  \
                                               {x: mnist.test.images, \
                                                y: mnist.test.labels, \
                                                keep_prob: 1.})))

def show_identity(graph):
    for i in graph.get_operations():
        print(i.name)

In [10]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint("Model/")) 
    graph = tf.graph_util.convert_variables_to_constants(sess, sess.graph_def, 
                                                         ["Y_hat", \
                                                          "cost",  \
                                                          "accuracy"])
    tf.train.write_graph(graph, "Model/", "graph.pb", as_text=False)

    test_mode = True
    if test_mode:
        graph = tf.get_default_graph()
        ckpt_test(graph)
        #show_identity(graph)

INFO:tensorflow:Restoring parameters from Model/lenet-100000
INFO:tensorflow:Froze 8 variables.
Converted 8 variables to const ops.
Testing Accuracy : 0.99


In [11]:
def pb_test(graph_def):
    x = tf.placeholder(tf.float32, [None, n_input], name = 'X')
    y = tf.placeholder(tf.float32, [None, n_classes], name = 'Y')
    keep_prob = tf.placeholder(tf.float32, name = "keep_prob") #dropout (keep probability)

    accuracy = tf.import_graph_def(graph_def, input_map =      \
                                   {"X:0": x, "Y:0": y,        \
                                    "keep_prob:0": keep_prob}, \
                                   return_elements = ["accuracy:0"])

    print("Testing Accuracy : %.2f" %(sess.run(accuracy, feed_dict =  \
                                               {x: mnist.test.images, \
                                                y: mnist.test.labels, \
                                                keep_prob: 1.})[0]))

In [12]:
with tf.Session() as sess:
    with open("Model/graph.pb", "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        pb_test(graph_def)

Testing Accuracy : 0.99


In [ ]:
!tensorboard --logdir=TensorBoard/

In [ ]:
!rm -r MNIST_data/ TensorBoard/ Model/